In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers, models

2024-11-18 11:08:44.170660: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-18 11:08:44.229311: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-18 11:08:44.281003: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731917324.332391    2833 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731917324.346235    2833 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 11:08:44.455179: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


W0000 00:00:1731920230.685488    2833 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
# Load dataset
tree_data_path = "./archive/tree.csv"
tree_data = pd.read_csv(tree_data_path)

# Select relevant columns
tree_data = tree_data[['Tree ID', 'Tree Type', 'Age (Estimated)']]

# Encode Tree ID and Tree Type into numerical values
encoder_id = LabelEncoder()
tree_data['Tree ID Encoded'] = encoder_id.fit_transform(tree_data['Tree ID'])

# Define X and y
X = tree_data['Tree ID'].values  # Input features (Tree IDs)
y = tree_data['Age (Estimated)'].values  # Output labels (Ages)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify the split
print("Training Data:", len(X_train), "Testing Data:", len(X_test))


Training Data: 38 Testing Data: 10


In [3]:
print(tree_data)

   Tree ID Tree Type  Age (Estimated)  Tree ID Encoded
0   AT3001     Amber               10                0
1   HT3001     Helwa               25                6
2   HT3002     Helwa               25                7
3   HT3003     Helwa               25                8
4   HT3004     Helwa               25                9
5   JT3001      Ajwa               25               31
6   KT3001   Sukkari               25               40
7   QT3001      Saqi               25               41
8   WT3001    Safawi               25               42
9   WT3002    Safawi               25               43
10  WT3003    Safawi               25               44
11  WT3004    Safawi               25               45
12  JT2001      Ajwa               17               10
13  JT2002      Ajwa               17               11
14  JT2003      Ajwa               17               12
15  JT2004      Ajwa               17               13
16  JT2005      Ajwa               17               14
17  JT2006

In [4]:
tree_type_counts = tree_data['Tree Type'].value_counts()
print(tree_type_counts)


Tree Type
Ajwa       30
Safawi      6
Barhi       5
Helwa       4
Amber       1
Saqi        1
Sukkari     1
Name: count, dtype: int64


In [5]:
tree_age_counts = tree_data['Age (Estimated)'].value_counts()
print(tree_age_counts)

Age (Estimated)
17    19
25    16
37     4
40     2
8      2
3      2
10     1
22     1
27     1
Name: count, dtype: int64


In [6]:
# Function to load all images for a given tree ID
def load_images_for_tree(tree_id):
    images = []
    for file_name in os.listdir(IMAGE_DIR):
        if file_name.startswith(tree_id) and file_name.endswith(".jpg"):
            # Load image, resize, and convert to array
            img_path = os.path.join(IMAGE_DIR, file_name)
            img = load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
            img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
            images.append(img_array)
    return np.stack(images) if images else None

In [7]:
# Build dataset of image tensors and labels
def build_image_dataset(tree_ids, ages):
    image_data = []
    age_labels = []
    for tree_id, age in zip(tree_ids, ages):
        images = load_images_for_tree(tree_id)
        if images is not None:
            image_data.append(images)
            age_labels.append(age)
    return np.array(image_data, dtype=object), np.array(age_labels)


In [8]:
# Parameters
IMG_HEIGHT, IMG_WIDTH = 512, 512  # Resize dimensions
IMAGE_DIR = "./archive/Palm Trees/Palm Trees/"  # Directory containing images
BATCH_SIZE = 32

In [9]:
# Prepare datasets
X_train_images, y_train_labels = build_image_dataset(X_train, y_train)
X_test_images, y_test_labels = build_image_dataset(X_test, y_test)

In [10]:
# Print dataset shapes
print(f"Training data shape: X=, {len(X_train_images)}, y={len(y_train_labels)}")
print(f"Testing data shape:, X={len(X_test_images)}, y={len(y_test_labels)}")

Training data shape: X=, 34, y=34
Testing data shape:, X=10, y=10


In [11]:
# Check the number of samples in the dataset
print("Number of training samples:", len(X_train_images))

# Iterate over a few samples to inspect their details
for idx, sample in enumerate(X_train_images[:5]):  # Inspect first 5 samples
    print(f"\nSample {idx + 1}:")
    
    # Check the type
    print("Type:", type(sample))  # Should be a numpy.ndarray
    
    # Check the shape
    print("Shape:", sample.shape)  # Should be (n_images, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
    
    # Check pixel value range
    print("Pixel Value Range: Min =", sample.min(), ", Max =", sample.max())




Number of training samples: 34

Sample 1:
Type: <class 'numpy.ndarray'>
Shape: (16, 512, 512, 3)
Pixel Value Range: Min = 0.0 , Max = 1.0

Sample 2:
Type: <class 'numpy.ndarray'>
Shape: (12, 512, 512, 3)
Pixel Value Range: Min = 0.0 , Max = 1.0

Sample 3:
Type: <class 'numpy.ndarray'>
Shape: (16, 512, 512, 3)
Pixel Value Range: Min = 0.0 , Max = 1.0

Sample 4:
Type: <class 'numpy.ndarray'>
Shape: (12, 512, 512, 3)
Pixel Value Range: Min = 0.0 , Max = 1.0

Sample 5:
Type: <class 'numpy.ndarray'>
Shape: (12, 512, 512, 3)
Pixel Value Range: Min = 0.0 , Max = 1.0


In [12]:
# Parameters
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 512, 512, 3  # Image dimensions
FEATURE_DIM = 256  # Feature vector size

# Define the CNN model for feature extraction
def build_feature_extractor():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.GlobalAveragePooling2D(),
        layers.Dense(FEATURE_DIM, activation='relu')  # Feature vector for each image
    ])
    return model

# Build the model for multiple images per tree
def build_tree_age_predictor():
    # Feature extractor
    feature_extractor = build_feature_extractor()
    
    # Input for multiple images of a tree
    image_inputs = layers.Input(shape=(None, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))  # (n_images, height, width, channels)
    
    # Process each image independently
    features = layers.TimeDistributed(feature_extractor)(image_inputs)  # (n_images, FEATURE_DIM)
    
    # Aggregate features (mean pooling)
    aggregated_features = layers.GlobalAveragePooling1D()(features)  # (FEATURE_DIM,)
    
    # Predict age
    outputs = layers.Dense(1)(aggregated_features)  # Regression output for age
    
    # Build the model
    model = models.Model(inputs=image_inputs, outputs=outputs)
    return model

# Instantiate the model
model = build_tree_age_predictor()

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])  # MSE for regression
model.summary()


/home/mohammed/anaconda3/envs/AI305_ML/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1731888380.270706    2984 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, None, 512, 512, │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, None, 256)      │       126,272 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 126,529 (494.25 KB)

 Trainable params: 126,529 (494.25 KB)

 Non-trainable params: 0 (0.00 B)